In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from itertools import product
import collections
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
import time
import pandas as pd
import json
from IPython.display import clear_output
import import_dataset as load
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle


# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
#Importation du data
train_path = "../dataset/train" 
test_path = "../dataset/test" 

train_dataset_X, train_dataset_Y = load.loadDataset(train_path)
test_dataset_X, test_dataset_Y = load.loadDataset(test_path)

print(train_dataset_X.shape)
print(train_dataset_Y.shape)

debut chargement image
debut chargement image
(12000, 224, 224)
(12000,)


In [16]:
print(test_dataset_X.shape)
print(test_dataset_Y.shape)

(8580, 224, 224)
(8580,)


In [3]:
# Encoding Y labels

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

train_dataset_Y = encoder.fit_transform(train_dataset_Y)
test_dataset_Y = encoder.fit_transform(test_dataset_Y)

print(np.unique(test_dataset_Y))


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]


In [4]:
train_dataset_X = train_dataset_X.reshape(12000, 1, 224, 224)
train_dataset_X  = torch.from_numpy(train_dataset_X).float()
train_dataset_Y = torch.from_numpy(train_dataset_Y).float()

print(train_dataset_X.shape)
print(train_dataset_Y.shape)

torch.Size([12000, 1, 224, 224])
torch.Size([12000])


In [5]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(12544, 12000)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [8]:
# defining the model
model = Net()
# model.to(device)
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
# /
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=12544, out_features=12000, bias=True)
  )
)


In [9]:
def train(epoch, train_dataset_X, train_dataset_Y):
    model.train()
    tr_loss = 0
    # getting the training set
    X_shuffle, y_shuffle = shuffle(train_dataset_X, train_dataset_Y, random_state=0)
    minibatch = 100
    for i in range (0, X_shuffle.shape[0], minibatch):
        # train_X, train_Y = Variable(train_dataset_X[i*stride:(i+1)*stride].cuda()), Variable(train_dataset_Y[i*stride:(i+1)*stride].cuda())
        train_X, train_Y = Variable(train_dataset_X[i:i + minibatch]), Variable(train_dataset_Y[i:i + minibatch])

    # converting the data into GPU format
#     if torch.cuda.is_available():
#         train_X = train_X.cuda()
#         train_Y = train_Y.cuda()
        
    # clearing the Gradients of the model parameters
        optimizer.zero_grad()
    
    # prediction for training and validation set
        output_train = model(train_X)

    # computing the training and validation loss
        loss_train = criterion(output_train, train_Y.long())
        train_losses.append(loss_train)

    # computing the updated weights of all the model parameters
        loss_train.backward()
        optimizer.step()
        tr_loss = loss_train.item()
        # i1 = i1 + 100
        # i2 = i2 + 100
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_train)
    return tr_loss

In [10]:
# defining the number of epochs
n_epochs = 25
# empty list to store training losses
train_losses = []
# training the model
for epoch in range(n_epochs):
    test = train(epoch, train_dataset_X, train_dataset_Y)
    print(test)

00
minibatch 3200
minibatch 3300
minibatch 3400
minibatch 3500
minibatch 3600
minibatch 3700
minibatch 3800
minibatch 3900
minibatch 4000
minibatch 4100
minibatch 4200
minibatch 4300
minibatch 4400
minibatch 4500
minibatch 4600
minibatch 4700
minibatch 4800
minibatch 4900
minibatch 5000
minibatch 5100
minibatch 5200
minibatch 5300
minibatch 5400
minibatch 5500
minibatch 5600
minibatch 5700
minibatch 5800
minibatch 5900
minibatch 6000
minibatch 6100
minibatch 6200
minibatch 6300
minibatch 6400
minibatch 6500
minibatch 6600
minibatch 6700
minibatch 6800
minibatch 6900
minibatch 7000
minibatch 7100
minibatch 7200
minibatch 7300
minibatch 7400
minibatch 7500
minibatch 7600
minibatch 7700
minibatch 7800
minibatch 7900
minibatch 8000
minibatch 8100
minibatch 8200
minibatch 8300
minibatch 8400
minibatch 8500
minibatch 8600
minibatch 8700
minibatch 8800
minibatch 8900
minibatch 9000
minibatch 9100
minibatch 9200
minibatch 9300
minibatch 9400
minibatch 9500
minibatch 9600
minibatch 9700
minibat

In [11]:
# Testing and predictions : 

test_dataset_X = test_dataset_X.reshape(8580, 1, 224, 224)
test_dataset_X  = torch.from_numpy(test_dataset_X).float()
test_dataset_Y = torch.from_numpy(test_dataset_Y).float()

print(test_dataset_X.shape)
print(test_dataset_Y.shape)

torch.Size([8580, 1, 224, 224])
torch.Size([8580])


In [16]:
# generating predictions for test set
output = model(test_dataset_X)

softmax = torch.exp(output)
prob = list(softmax)
predictions = np.argmax(prob, axis=0)

# accuracy on training set
accuracy_score(train_y, predictions)

RuntimeError: bool value of Tensor with more than one value is ambiguous